# LCT-AGI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain-tools/blob/main/examples/sample.ipynb)

In [1]:
%%capture
!pip install langchain
!pip install openai

In [6]:
import os

from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, tool
from langchain.tools import BaseTool

In [11]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"


### サンプル

In [12]:
class MultiplicationTool(BaseTool):
    name = "MultiplicationTool"
    description = """used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."""

    def _run(self, query: str) -> str:
      """Use the tool."""
      a, b = query.split(",")
      c = int(a) * int(b)
      result = c

      return result 

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [13]:
tools = [MultiplicationTool()]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [14]:
agent.run('38 times 47?')



> Entering new AgentExecutor chain...
 I need to multiply two numbers
Action: MultiplicationTool
Action Input: 38,47
Observation: 1786
Thought: I now know the final answer
Final Answer: 38 times 47 is 1786.

> Finished chain.


'38 times 47 is 1786.'

In [ ]:
llm('38 times 47?')

### 参照するLLM

In [8]:
#input = 'trainig XGBoost'
input = 'training LightGBM'
exec_list = [
    'training LightGBM'
]

In [9]:
def deciding_agent(input_, exec_list_):
    print("deciding agent")
    
    decide_llm = OpenAI(temperature=0,model_name="gpt-3.5-turbo")

    i = ''
    for i in exec_list:
        i = i + ','
    i = '[' + i + ']'

    prompt = """
    You are an agent that determines if the input task is executable. 
    All you can do is to be included in {exec_list}. 
    Answer "Yes." if you can perform the task, or "No." if you cannot.
    ------
    The entered task is:{input}
    ------
    """.format(exec_list = i,input = input_)

    return decide_llm(prompt)

In [10]:
deciding_agent(input, exec_list)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:672: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'Yes'

### 道具をつくるLLM

In [15]:
def toolmaking_agent(input_):
    print("toolmaking agent")

    toolmaking_llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")


    prompt = """
    Create a python function that can execute {input} with a single string as input.
    The code below is the code created with the input "multiply two numbers".
    The output is only the code of the created python.
    ```
    from dataclasses import dataclass

    @dataclass
    class NewTool(BaseTool):
        name = "MultiplicationTool"
        description = "used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."

        def _run(self, query: str) -> str:
            "Use the tool."
            a, b = query.split(",")
            c = int(a) * int(b)
            result = c

        return result 

        async def _arun(self, query: str) -> str:
            "Use the tool asynchronously."
            raise NotImplementedError("BingSearchRun does not support async")
    ```
    """.format(input = input_)

    code = toolmaking_llm(prompt)


    file = open('/content/new_tool.py', mode='w')
    file.write(code)
    file.close()

    return code

In [24]:
input = 'Output multiples of 3 up to 20'
toolmaking_agent(input)

toolmaking agent


'finish'

### 実行するLLM

In [ ]:
def executing_agent(input_, tools_):
    print("executing agent")

    excute_llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")

    agent = initialize_agent(tools_, excute_llm, agent="zero-shot-react-description", verbose=True)

    agent.run(input_)

    return

### 全体

In [ ]:
def main(input_, exec_list_, tools_):
    
    output = deciding_agent(input_, exec_list_)

    if output == "Yes":
        print('Execute it as it is executable without creating a tool.')
        executing_agent(input_, tools_)

    elif output == "No":
        print('It is necessary to create a tool, so run it after creating the tool.')
        new_tool = toolmaking_agent(input_)

        exec(new_tool)

        tools_.append(NewTool())

        executing_agent(input_, tools_)
    
    return